<a href="https://colab.research.google.com/github/samueleallen/Scraping-Val-Data/blob/main/scraping_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from google.colab import files
import time

In [ ]:
# scrape team urls from standings page
standings_url = 'https://www.vlr.gg/vct-2024/standings'
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

In [ ]:
# Locate team urls
standings_table = soup.select('div.eg-standing-container')[0]
links = standings_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/team/' in l]
# Format link
team_urls = [f'https://www.vlr.gg{l}' for l in links]

In [ ]:
# CSV file path for team map stats
csv_file_path = '/content/team_map_stats.csv'

# List to hold all team data
all_team_data = []

for team_url in team_urls: # just get rid of [:1] to fix loop later, did this so runtime is not too long
  data = requests.get(team_url)

  # Scrape stats section data from team page
  soup = BeautifulSoup(data.text)
  links = soup.find_all('a')
  links = [l.get("href") for l in links]
  links = [l for l in links if l and 'team/stats' in l]

  for link in links:
    data = requests.get(f"https://www.vlr.gg{link}")
    soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
    table = soup.find('table', class_='wf-table mod-team-maps') # find table on webpage
    # Find the h1 element with class 'wf-title' and get the text
    team_name = soup.find('h1', class_='wf-title').get_text(strip=True)
    # Use pandas to read the table
    stats = pd.read_html(str(table))[0]
    # Filter rows to only be rows that start with specific map names
    map_names = ["Sunset", "Bind", "Haven", "Split", "Ascent", "Icebox", "Breeze", "Fracture", "Pearl", "Lotus", "Abyss"]
    filtered_stats = stats[stats.iloc[:, 0].str.startswith(tuple(map_names), na=False)]
    filtered_stats = filtered_stats.drop(columns=["Agent Compositions", "Expand"])
    filtered_stats['Team'] = team_name
    all_team_data.append(filtered_stats)
    time.sleep(1)
  # After the loop finishes, concatenate all the team data into a single DataFrame
if all_team_data:
    full_team_stats = pd.concat(all_team_data, ignore_index=True)

    # Write the combined team stats to the CSV file (overwrite the file)
    full_team_stats.to_csv(csv_file_path, mode='w', header=True, index=False)

    # After the loop finishes, download the file
    # files.download(csv_file_path)

<ipython-input-4-e234b8de7fbe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]
<ipython-input-4-e234b8de7fbe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]
<ipython-input-4-e234b8de7fbe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]
<ipython-input-4-e234b8de7fbe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]
<ipython-input-4-e234b8de7fbe>:2

In [ ]:
# CSV file path for player stats
csv_file_path = '/content/player_stats.csv'

# List to hold all player data
all_player_data = []

# Fetches individual player data from their entire career
for team_url in team_urls:
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'player' in l]

    # Convert to full player URLs and filter to only players, not coaching & staff
    player_urls = [f"https://www.vlr.gg{l}" for l in links]
    player_stats_urls = [f"{l}/?timespan=all" for l in player_urls[:5]]

    # Loop through each player and collect their data
    for player_url in player_stats_urls:
        try:
            data = requests.get(player_url)
            soup = BeautifulSoup(data.text)
            table = soup.find('table', class_='wf-table')

            if table is None:
                print(f"No table found for {player_url}")
                continue  # Skip this player and move to the next

            # Read table with pandas
            player_stats = pd.read_html(str(table))[0]

            # Extract player name from URL
            player_name = player_url.split("/")[-2]  # Gets the player name from the URL

            # Extract agent names from images
            agent_names = []
            for img_tag in table.find_all('img'):
                img_src = img_tag.get('src')
                agent_name = os.path.splitext(os.path.basename(img_src))[0]
                agent_names.append(agent_name)

            # Add extracted agent names and player name column
            player_stats['Agent'] = agent_names
            player_stats['Player'] = player_name  # Append player name column
            player_stats = player_stats.drop(columns=["Unnamed: 0"])

            # Append player stats to the all_player_data list
            all_player_data.append(player_stats)

        except Exception as e:
            print(f"Error processing {player_url}: {e}")

# After the loop finishes, concatenate all the player data into a single DataFrame
if all_player_data:
    full_stats = pd.concat(all_player_data, ignore_index=True)

    # Write the combined player stats to the CSV file (overwrite the file)
    full_stats.to_csv(csv_file_path, mode='w', header=True, index=False)


<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipy

No table found for https://www.vlr.gg/player/40862/benedict/?timespan=all
No table found for https://www.vlr.gg/player/31688/impression/?timespan=all
No table found for https://www.vlr.gg/390246/bleed-announce-roster-overhaul-bench-four-players/?timespan=all


<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-5-ea1c1a905591>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipy

In [ ]:
# CSV file path for player stats
csv_file_path = '/content/player_stats_90days.csv'

# List to hold all player data
all_player_data = []

# Fetches individual player data from their entire career
for team_url in team_urls:
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'player' in l]

    # Convert to full player URLs and filter to only players, not coaching & staff
    player_urls = [f"https://www.vlr.gg{l}" for l in links]
    player_stats_urls = [f"{l}/?timespan=90d" for l in player_urls[:5]]

    # Loop through each player and collect their data
    for player_url in player_stats_urls:
        try:
            data = requests.get(player_url)
            soup = BeautifulSoup(data.text)
            table = soup.find('table', class_='wf-table')

            if table is None:
                print(f"No table found for {player_url}")
                continue  # Skip this player and move to the next

            # Read table with pandas
            player_stats = pd.read_html(str(table))[0]

            # Extract player name from URL
            player_name = player_url.split("/")[-2]  # Gets the player name from the URL

            # Extract agent names from images
            agent_names = []
            for img_tag in table.find_all('img'):
                img_src = img_tag.get('src')
                agent_name = os.path.splitext(os.path.basename(img_src))[0]
                agent_names.append(agent_name)

            # Add extracted agent names and player name column
            player_stats['Agent'] = agent_names
            player_stats['Player'] = player_name  # Append player name column
            player_stats = player_stats.drop(columns=["Unnamed: 0"])

            # Append player stats to the all_player_data list
            all_player_data.append(player_stats)

        except Exception as e:
            print(f"Error processing {player_url}: {e}")

# After the loop finishes, concatenate all the player data into a single DataFrame
if all_player_data:
    full_stats = pd.concat(all_player_data, ignore_index=True)

    # Write the combined player stats to the CSV file (overwrite the file)
    full_stats.to_csv(csv_file_path, mode='w', header=True, index=False)

<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]


No table found for https://www.vlr.gg/player/1129/rossy/?timespan=90d


<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipy

No table found for https://www.vlr.gg/player/40862/benedict/?timespan=90d
No table found for https://www.vlr.gg/player/31688/impression/?timespan=90d
No table found for https://www.vlr.gg/390246/bleed-announce-roster-overhaul-bench-four-players/?timespan=90d


<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipy

No table found for https://www.vlr.gg/player/25494/shr1mp/?timespan=90d


<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipy

No table found for https://www.vlr.gg/player/37020/xiyiji/?timespan=90d


<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-6-b0991c6d8f35>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipy

In [ ]:
# I have all-time player stats, past 90 days player stats, team map stats, now we need match up stats for teams 😸
# Ex: fnatic vs sentinels, sentinels has won x out of y matchups
matches_url = 'https://www.vlr.gg/matches'
data = requests.get(matches_url)
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'page' in l]
url = "https://www.vlr.gg/matches/results"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Find all page number links
page_links = soup.find_all("a", class_="btn mod-page")
# Page_links holds a list of HTML <a> elements beautiful soup found
# Now extract the page numbers
page_numbers = [int(link.text) for link in page_links if link.text.isdigit()]
outer_list = []

# Get the max page number
max_page = max(page_numbers)

In [ ]:
for page in range(1, max_page + 1): # For loop starts from page 1 ends at max page
    url = f"https://www.vlr.gg/matches/results/?page={page}"
    data = requests.get(url)
    soup = BeautifulSoup(data.text, "html.parser")
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    # Filter links to only include recent years, champions tour matches, and not acension champions tour matches
    links = [l for l in links if l and 'champions-tour' in l and ('2023' in l or '2024' in l or '2025' in l) and 'ascension' not in l and 'challengers' not in l]
    all_time_query_string = "?game=all&tab=overview"
    matches = [f"https://www.vlr.gg{l}{all_time_query_string}" for l in links]
    # Append each link to a different list so all elements stay in one list throughout the 550+ loops
    outer_list.extend(matches)

In [ ]:
import re

# CSV file path for overall game stats
csv_file_path = '/content/overall_game_stats.csv'

# List to hold all player data
all_game_stats = []

# Outer list holds every vct match from 2023-current time
for match in outer_list:
    try:
        # First, determine winner and loser by going to last match tab
        # and extracting final scores
        data = requests.get(match)
        soup = BeautifulSoup(data.text)
        divs_list = soup.find_all('div', class_='vm-stats-gamesnav-item js-map-switch')
        all_match_tabs = []

        # Find all div elements with the given class
        for i in range(len(divs_list)):
          div = divs_list[i]
          # Extract and print the 'data-href' attributes
          data_href = div.get('data-href')
          game_id = div.get("data-game-id")
          if data_href and game_id:
            final_url = f"https://www.vlr.gg{data_href}&game={game_id}&tab=overview"
            overall_url = f"https://www.vlr.gg{data_href}&game={game_id}&game=all&tab=overview"
            all_match_tabs.append(final_url)

        final_match_tab = all_match_tabs[-1]
        # We have to scrape from final map played to see who won last map to get our winner
        # idea is we get last map score and the first score should represent the first team
        # in our team list, second score will be our second team in the teams list
        # and with that we find who won/lost
        scores = []
        all_scores = soup.find_all('div', class_='score')  # Get all score elements

        # Extract text from each score div
        for score in all_scores:
            scores.append(score.get_text(strip=True))

        # only keep last score from each team to see who won.
        scores = [scores[-2], scores[-1]]

        # Secondly, we fetch overall match data
        data = requests.get(overall_url)
        soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page

        table = soup.find_all('table', class_='wf-table-inset mod-overview')

        team_names = [team.get_text(strip=True) for team in soup.find_all('div', class_='wf-title-med')]

        # Extract the year from the event name
        event_string = soup.find('div', style='font-weight: 700;').get_text(strip=True)  # Find the event string
        match_year = re.search(r'\d{4}', event_string)  # Find the year in the event string
        if match_year:
            year = match_year.group(0)
        else:
            year = None  # If no year is found, set it as None

        # Extract the winner and loser scores
        winner_score = soup.find('span', class_='match-header-vs-score-winner').get_text(strip=True)
        loser_score = soup.find('span', class_='match-header-vs-score-loser').get_text(strip=True)

        # Extract the match date from the div with class 'moment-tz-convert'
        date_string = soup.find('div', class_='moment-tz-convert')['data-utc-ts']  # Extract the full timestamp
        date = pd.to_datetime(date_string).strftime('%Y-%m-%d')  # Convert to date format (YYYY-MM-DD)

        result = 'Win'
        final_score = f"{winner_score}-{loser_score}"

        # Use pandas to read the table
        for j in range(2, 4):
            try:
                all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
                # print(all_map_stats)
                all_map_stats = all_map_stats.drop(columns=["Unnamed: 1"])

                # Clean the data by keeping only the first number in each cell
                all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)

                # Add the year and date to the DataFrame
                all_map_stats['date'] = date
                all_map_stats['Team'] = team_names[j-2]
                all_map_stats['vs Team'] = team_names[3-j]

                # Set the result column based on the winning team. If the first score (which is team 1) is greater than second score (team 2)
                # then the first team won
                if (int(scores[0]) > int(scores[1])):
                  # if first score (team 1) has a higher score than second score (team 2), then team 1 won game
                  if j == 2:
                    # if team 1, set result to win
                    all_map_stats['result'] = 'Win'
                    all_map_stats['final_score'] = f"'{winner_score} : {loser_score}'"  # Winner score comes first
                  else:
                    # if team 2, set result to loss
                    all_map_stats['result'] = 'Loss'
                    all_map_stats['final_score'] = f"'{loser_score} : {winner_score}'"  # Loser score comes first
                else:
                  # if first score (Team 1) is less than second score in list (Team 2), then team 2 won game
                  if j == 2:
                    # if team 1, set result to loss
                    all_map_stats['result'] = 'Loss'
                    all_map_stats['final_score'] = f"'{loser_score} : {winner_score}'"  # Loser score comes first
                  else:
                    # if team 2, set result to win
                    all_map_stats['result'] = 'Win'
                    all_map_stats['final_score'] = f"'{winner_score} : {loser_score}'"  # Winner score comes first

                # Rename columns as needed
                all_map_stats = all_map_stats.rename(columns={'Unnamed: 0': 'Player', '+/–': '+/- K/D', '+/–.1': '+/- FK/FD'})

                print(all_map_stats)

                all_game_stats.append(all_map_stats)

            except Exception as e:
                print(f"Error processing match data for {match}, table {j}: {e}")
                continue  # Skip this iteration and continue with the next table

    except Exception as e:
        print(f"Error processing match {match}: {e}")
        continue  # Skip this iteration and continue with the next match

# After the loop finishes, concatenate all the player data into a single DataFrame
if all_game_stats:
    full_stats = pd.concat(all_game_stats, ignore_index=True)

    # Write the combined game stats to the CSV file (overwrite the file)
    full_stats.to_csv(csv_file_path, mode='w', header=True, index=False)

<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables


    Player  R2.0  ACS    K   D   A +/- K/D KAST  ADR  HS%  FK  FD +/- FK/FD  \
0  jawgemo  1.18  243  103  89  44     +14  78%  160  28%  12  20        -8   
1    trent  1.11  224   92  79  40     +13  75%  145  29%  14   9        +5   
2    valyn  1.02  210   94  82  51     +12  77%  138  25%  10  10         0   
3     leaf  0.81  190   79  86  31      -7  75%  119  21%  13  15        -2   
4   JonahP  0.92  162   74  87  42     -13  75%  105  28%   7  11        -4   

         date        Team vs Team result final_score  
0  2025-03-02  G2 Esports      T1   Loss     '2 : 3'  
1  2025-03-02  G2 Esports      T1   Loss     '2 : 3'  
2  2025-03-02  G2 Esports      T1   Loss     '2 : 3'  
3  2025-03-02  G2 Esports      T1   Loss     '2 : 3'  
4  2025-03-02  G2 Esports      T1   Loss     '2 : 3'  
   Player  R2.0  ACS   K   D   A +/- K/D KAST  ADR  HS%  FK  FD +/- FK/FD  \
0    stax  1.01  184  82  84  35      -2  75%  132  29%   6   3        +3   
1  Meteor  0.96  219  94  96  30      -2 

<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)


Error processing match https://www.vlr.gg/450589/team-international-vs-team-thailand-champions-tour-2025-masters-bangkok-main-event?game=all&tab=overview: list index out of range


<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = 

...

      Player  R2.0  ACS   K   D   A +/- K/D KAST  ADR  HS% FK  FD +/- FK/FD  \
0     YuChEn  0.95  189  40  42  20      -2  76%  125  34%  3   7        -4   
1     Yuicaw  1.07  190  42  40   7      +2  66%  132  22%  3  12        -9   
2      AAAAY  0.88  202  39  49  20     -10  75%  129  31%  1   6        -5   
3  nizhaoTZH  0.88  162  35  42  16      -7  63%   89  23%  4   5        -1   
4     BerLIN  0.69  182  34  47  23     -13  76%  123  28%  7  11        -4   

         date             Team       vs Team result final_score  
0  2023-02-14  FunPlus Phoenix  Karmine Corp   Loss     '1 : 2'  
1  2023-02-14  FunPlus Phoenix  Karmine Corp   Loss     '1 : 2'  
2  2023-02-14  FunPlus Phoenix  Karmine Corp   Loss     '1 : 2'  
3  2023-02-14  FunPlus Phoenix  Karmine Corp   Loss     '1 : 2'  
4  2023-02-14  FunPlus Phoenix  Karmine Corp   Loss     '1 : 2'  
    Player  R2.0  ACS   K   D   A +/- K/D KAST  ADR  HS%  FK FD +/- FK/FD  \
0     sh1n  1.59  279  57  38  19     +19  75%  170 

<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = 

    Player  R2.0  ACS   K   D   A +/- K/D KAST  ADR  HS% FK FD +/- FK/FD  \
0    takej  1.31  197  13  13   5       0  60%  120  42%  2  1        +1   
1    Reita  0.76  208  25  30   8      -5  57%  137  44%  3  8        -5   
2   Anthem  0.83  159  19  29   6     -10  68%  109  32%  3  2        +1   
3    xnfri  0.77  151  19  30   6     -11  57%  107  19%  1  2        -1   
4  Suggest  0.62  155  21  31  14     -10  65%   91  30%  4  5        -1   
5  Seoldam  0.32  129   7  16   2      -9  53%   80  31%  2  4        -2   

         date                Team        vs Team result final_score  
0  2023-02-13  DetonatioN FocusMe  Giants Gaming   Loss     '0 : 2'  
1  2023-02-13  DetonatioN FocusMe  Giants Gaming   Loss     '0 : 2'  
2  2023-02-13  DetonatioN FocusMe  Giants Gaming   Loss     '0 : 2'  
3  2023-02-13  DetonatioN FocusMe  Giants Gaming   Loss     '0 : 2'  
4  2023-02-13  DetonatioN FocusMe  Giants Gaming   Loss     '0 : 2'  
5  2023-02-13  DetonatioN FocusMe  Giants Gamin

<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-38-84735e237cf9>:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-38-84735e237cf9>:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = 

       Player  R2.0  ACS   K   D   A +/- K/D KAST  ADR  HS%  FK  FD +/- FK/FD  \
0       trexx  1.70  275  46  27   9     +19  76%  181  32%   3   4        -1   
1     sheydos  1.26  158  27  24   6      +3  70%  107  24%   2   4        -2   
2      Wolfen  0.74  187  28  38   7     -10  61%  123  33%  10  13        -3   
3      starxo  0.75  123  17  29  22     -12  70%   79  18%   0   6        -6   
4  koldamenta  0.72  110  17  34  19     -17  63%   75  21%   1   3        -2   

         date Team      vs Team result final_score  
0  2023-02-13  KOI  NRG Esports   Loss     '0 : 2'  
1  2023-02-13  KOI  NRG Esports   Loss     '0 : 2'  
2  2023-02-13  KOI  NRG Esports   Loss     '0 : 2'  
3  2023-02-13  KOI  NRG Esports   Loss     '0 : 2'  
4  2023-02-13  KOI  NRG Esports   Loss     '0 : 2'  
     Player  R2.0  ACS   K   D   A +/- K/D KAST  ADR  HS%  FK FD +/- FK/FD  \
0  crashies  1.44  220  36  27  17      +9  74%  139  27%   5  4        +1   
1       s0m  1.25  227  36  26  12     

In [ ]:
# minor error, table gets confusing. Elements of three which go from these stats categories "all -> attack -> defend"
# Need to get data href for each map played
# And need to get each data-game id
# The next bit of code is to try to get each map's stats for each map played in a game. Previously,
# we got the stats for all the maps combined but we want to see how they performed on each map.
data = requests.get(outer_list[0]) # will need a for loop later to go through every element of outer list 🐈

In [ ]:
# Locate team urls
# maps = soup.select('vm-stats-gamesnav-item js-map-switch')[0]
soup = BeautifulSoup(data.text, 'html.parser')
# need amount of maps played for max index range
divs = soup.find_all('div', class_='vm-stats-gamesnav-item js-map-switch')
all_match_tabs = []

In [ ]:
# Find all div elements with the given class
for i in range(0, len(divs)):
  divs = soup.find_all('div', class_='vm-stats-gamesnav-item js-map-switch')[i]
  # Extract and print the 'data-href' attributes
  data_href = divs.get('data-href')
  game_id = divs.get("data-game-id")
  if data_href and game_id:
    final_url = f"https://www.vlr.gg{data_href}&game={game_id}&tab=overview"
    all_match_tabs.append(final_url)
    print(all_match_tabs)

['https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=1&game=202792&tab=overview']
['https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=1&game=202792&tab=overview', 'https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=2&game=202793&tab=overview']
['https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=1&game=202792&tab=overview', 'https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=2&game=202793&tab=overview', 'https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=3&game=202794&tab=overview']
['https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=1&game=202792&tab=overview', 'https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2025-masters-bangkok-gf/?map=2&game=202793&tab=overview', 'https://www.vlr.gg/449011/g2-esports-vs-t1-champions-tour-2

In [ ]:
for i in range(0, (len(all_match_tabs))): # outer range should be len(all_match_tabs) when done which is 5 maps current, usually 3
  data = requests.get(all_match_tabs[i])
  soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
  table = soup.find_all('table', class_='wf-table-inset mod-overview')
  individual_map_stats_team = pd.read_html(str(table))[i]
  individual_map_stats_team = individual_map_stats_team.drop(columns=["Unnamed: 1"])
  print(individual_map_stats_team.head())


<ipython-input-14-08d5da85be18>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[i]


   Unnamed: 0            R2.0          ACS        K            D       A  \
0    trent G2  1.55 1.75 1.16  296 321 247  17 13 4   / 11 7 4 /  10 5 5   
1     leaf G2  1.55 1.59 1.48  279 276 287  21 13 8   / 10 6 4 /   2 2 0   
2  jawgemo G2  1.16 0.88 1.72  236 232 245  16 10 6  / 12 10 2 /   7 4 3   
3    valyn G2  0.91 0.68 1.39  167 153 198   10 6 4   / 11 9 2 /   8 5 3   
4   JonahP G2  0.87 0.73 1.13  123 101 170    9 5 4   / 10 7 3 /   4 2 2   

         +/–          KAST          ADR          HS%     FK     FD     +/–.1  
0    +6 +6 0   72% 75% 67%  213 224 190  18% 18% 18%  3 2 1  1 1 0  +2 +1 +1  
1  +11 +7 +4  89% 83% 100%  159 172 135  19% 21% 14%  3 1 2  1 1 0   +2 0 +2  
2    +4 0 +4   72% 67% 83%  133 129 141  24% 23% 25%  1 1 0  3 3 0   -2 -2 0  
3   -1 -3 +2   78% 75% 83%  122 111 144  28% 33% 20%  1 0 1  3 2 1   -2 -2 0  
4   -1 -2 +1   83% 92% 67%    82 69 109  69% 71% 67%  2 1 1  0 0 0  +2 +1 +1  


<ipython-input-14-08d5da85be18>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[i]


  Unnamed: 0            R2.0          ACS        K            D      A  \
0     iZu T1  1.07 1.17 1.02  209 230 200   14 5 9   / 13 4 9 /  2 0 2   
1    BuZz T1  0.99 1.34 0.82  241 308 208   13 6 7   / 14 5 9 /  3 1 2   
2  Meteor T1  0.71 0.01 1.20   184 18 267  11 0 11  / 16 6 10 /  3 1 2   
3  Sylvan T1  0.68 0.57 0.73  159 139 170    8 2 6   / 15 6 9 /  9 5 4   
4    stax T1  0.54 0.37 0.62   116 76 137    8 2 6  / 15 5 10 /  2 0 2   

        +/–         KAST          ADR          HS%     FK     FD     +/–.1  
0   +1 +1 0  67% 50% 75%  139 165 126  48% 67% 39%  1 0 1  1 1 0   0 -1 +1  
1  -1 +1 -2  67% 67% 67%  162 179 154  17% 18% 17%  3 1 2  4 2 2   -1 -1 0  
2  -5 -6 +1  44% 17% 58%   120 18 171   31% 0% 35%  2 0 2  3 2 1  -1 -2 +1  
3  -7 -4 -3  78% 67% 83%    99 90 103  26% 25% 26%  2 0 2  1 0 1   +1 0 +1  
4  -7 -3 -4  56% 50% 58%     82 54 96  50% 50% 50%  0 0 0  1 0 1   -1 0 -1  


<ipython-input-14-08d5da85be18>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[i]


   Unnamed: 0            R2.0          ACS          K             D         A  \
0  jawgemo G2  1.18 1.20 1.19  243 262 228  103 59 44  / 89 50 39 /  44 21 23   
1    trent G2  1.11 1.25 1.17  224 193 243   92 42 50  / 79 43 36 /  40 22 18   
2    valyn G2  1.02 1.28 1.14  210 205 226   94 49 45  / 82 48 34 /  51 27 24   
3     leaf G2  0.81 1.06 0.93  190 160 210   79 36 43  / 86 45 41 /  31 13 18   
4   JonahP G2  0.92 0.83 0.88  162 166 166   74 40 34  / 87 46 41 /  42 17 25   

          +/–         KAST          ADR          HS%       FK       FD  \
0   +14 +9 +5  78% 77% 79%  160 165 154  28% 29% 26%   12 9 3  20 18 2   
1  +13 -1 +14  75% 72% 79%  145 129 164  29% 26% 31%   14 6 8    9 7 2   
2  +12 +1 +11  77% 77% 77%  138 127 150  25% 26% 24%   10 5 5   10 6 4   
3    -7 -9 +2  75% 71% 81%  119 107 133  21% 28% 14%  13 3 10   15 6 9   
4   -13 -6 -7  75% 78% 72%   105 110 98  28% 30% 25%    7 1 6   11 3 8   

      +/–.1  
0  -8 -9 +1  
1  +5 -1 +6  
2   0 -1 +1  
3  -2 -3 +1 

<ipython-input-14-08d5da85be18>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[i]


  Unnamed: 0            R2.0          ACS         K             D         A  \
0    stax T1  1.01 0.99 1.00  184 187 190  82 39 43  / 84 41 43 /  35 17 18   
1  Meteor T1  0.96 1.05 1.00  219 225 227  94 43 51  / 96 47 49 /  30 13 17   
2  Sylvan T1  0.94 1.04 0.99  191 166 215  82 33 49  / 84 40 44 /  65 33 32   
3     iZu T1  0.94 0.89 0.91  190 193 182  78 37 41  / 87 41 46 /  42 23 19   
4    BuZz T1  0.75 1.00 0.88  205 189 214  84 36 48  / 96 47 49 /   22 9 13   

          +/–         KAST          ADR          HS%       FK       FD  \
0     -2 -2 0  75% 75% 74%  132 134 131  29% 28% 31%    6 1 5    3 0 3   
1    -2 -4 +2  65% 61% 68%  153 161 147  31% 32% 30%  18 7 11   13 9 4   
2    -2 -7 +5  77% 70% 83%   111 94 126  24% 24% 23%   12 4 8    4 1 3   
3    -9 -4 -5  74% 70% 77%  122 122 121  27% 23% 31%   10 3 7   11 6 5   
4  -12 -11 -1  69% 58% 78%  128 118 136  19% 20% 18%  19 10 9  25 16 9   

      +/–.1  
0  +3 +1 +2  
1  +5 -2 +7  
2  +8 +3 +5  
3  -1 -3 +2  
4   -6 -6 

<ipython-input-14-08d5da85be18>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[i]
